In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=4ed3d9d2502d0d9fb27bd3c4630439251f4caa70dd024afaa636a847072495de
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 4.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F3 '''
def F3(X):
    f = bn.F3()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.11160 ,random_state=0 , l1_ratio= 0.9799, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 4.96 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_3_1000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_3_200Samples.csv').iloc[:,-1]
path = "train_latent_50D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_50D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,26):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 25000)

n_obs , dim =  30, 25
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,75])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=838996, Mon Apr 13 13:03:30 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 6.480118784517267e+03 1.0e+00 1.23e-01  1e-01  1e-01 0:00.0
    2     26 6.474132280182680e+03 1.1e+00 1.22e-01  1e-01  1e-01 0:00.0
    3     39 6.462539499017828e+03 1.2e+00 1.23e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6488.58091598] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6480.49756621] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 -7.718260552439606e+02 1.8e+00 9.36e-03  8e-03  1e-02 0:01.1
  200   2600 -7.900276351738030e+02 2.1e+00 1.20e-04  8e-05  1e-04 0:02.2
  300   3900 -7.900296067970930e+02 2.5e+00 2.12e-06  1e-06  2e-06 0:03.3
  400   5200 -7.900296071266612e+02 2.7e+00 2.22e-08  1e-08  2e-08 0:04.4
  492   6396 -7.900296071267157e+02 3.2e+00 1.81e-09  7e-10  1e-09 0:05.4
termination on tolfun=1e-11 (Mon Apr 13 13:03:37 2020)
final/bestever f-value = -7.900296e+02 -7.900296e+02
incumbent solution: [-0.00982798  0.07743961 -0.05328936 -0.04366306 -0.25058487  0.22432777
 -0.00688878 -0.00087847 ...]
std deviations: [9.89875721e-10 9.77977452e-10 7.48054957e-10 9.58124068e-10
 9.22095682e-10 9.24060186e-10 1.03340631e-09 9.31953044e-10 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=846444, Mon Apr 13 13:03:37 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 6.134101434611265e+03 1.0e+00 1.20e-01  1e-01  1e-01 0:00.0
    2     26 6.1296

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6138.24428102] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6141.29949873] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 5.026481706648383e+02 1.8e+00 9.47e-03  8e-03  9e-03 0:00.9
  200   2600 4.911994567889224e+02 2.0e+00 1.76e-04  1e-04  1e-04 0:01.8
  300   3900 4.911971297457221e+02 2.0e+00 2.33e-06  1e-06  2e-06 0:02.7
  400   5200 4.911971294213545e+02 2.2e+00 2.89e-08  1e-08  2e-08 0:03.6
  500   6500 4.911971294213045e+02 2.6e+00 4.86e-09  2e-09  3e-09 0:04.5
  600   7800 4.911971294212990e+02 3.2e+00 3.62e-09  1e-09  2e-09 0:05.4
  700   9100 4.911971294212972e+02 3.7e+00 2.10e-09  7e-10  1e-09 0:06.3
  800  10400 4.911971294212963e+02 4.3e+00 1.25e-09  4e-10  8e-10 0:07.2
  900  11700 4.911971294212963e+02 5.2e+00 9.43e-10  3e-10  7e-10 0:08.0
 1000  13000 4.911971294212944e+02 5.7e+00 1.27e-09  4e-10  8e-10 0:08.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  14300 4.911971294212963e+02 5.9e+00 1.43e-09  4e-10  9e-10 0:09.8
 1200  15600 4.911971294212999e+02 6.9e+00 3.77e-10  1e-10  2e-10 0:10.7
 1300  16900 4.911971294212990e+02 9.6e+00 4.96e-10 

...

  100   1300 -7.483462574153646e+02 1.9e+00 1.08e-02  9e-03  1e-02 0:01.1
  200   2600 -7.900197943811809e+02 2.1e+00 2.08e-04  1e-04  2e-04 0:02.2
  300   3900 -7.900296029922756e+02 2.3e+00 4.96e-06  3e-06  4e-06 0:03.3
  400   5200 -7.900296071263210e+02 2.5e+00 6.77e-08  3e-08  5e-08 0:04.4
  500   6500 -7.900296071267130e+02 2.9e+00 2.54e-09  1e-09  1e-09 0:05.4
  518   6734 -7.900296071267148e+02 3.0e+00 2.58e-09  1e-09  2e-09 0:05.6
termination on tolfun=1e-11 (Mon Apr 13 13:15:06 2020)
final/bestever f-value = -7.900296e+02 -7.900296e+02
incumbent solution: [-0.00982798  0.07743961 -0.05328936 -0.04366306 -0.25058487  0.22432777
 -0.00688878 -0.00087847 ...]
std deviations: [1.37563560e-09 1.34866905e-09 1.11210505e-09 1.32790974e-09
 1.14684480e-09 1.27600789e-09 1.32608769e-09 1.36674262e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=907191, Mon Apr 13 13:15:06 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 6.139

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6139.47856802] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6139.40114554] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 5.005257021315001e+02 1.7e+00 8.08e-03  7e-03  8e-03 0:00.9
  200   2600 4.911986107775892e+02 1.9e+00 1.28e-04  8e-05  1e-04 0:01.8
  300   3900 4.911971296309857e+02 2.2e+00 1.44e-06  7e-07  1e-06 0:02.7
  400   5200 4.911971294213308e+02 2.3e+00 2.00e-08  9e-09  1e-08 0:03.6
  500   6500 4.911971294213045e+02 2.8e+00 4.54e-09  2e-09  3e-09 0:04.5
  600   7800 4.911971294212981e+02 3.2e+00 3.02e-09  1e-09  2e-09 0:05.4
  700   9100 4.911971294213008e+02 3.7e+00 2.57e-09  1e-09  1e-09 0:06.3
  800  10400 4.911971294212990e+02 4.5e+00 9.52e-10  3e-10  6e-10 0:07.2
  884  11492 4.911971294212999e+02 4.9e+00 7.29e-10  3e-10  4e-10 0:07.9
termination on tolfun=1e-11 (Mon Apr 13 13:15:16 2020)
final/bestever f-value = 4.911971e+02 4.911971e+02
incumbent solution: [-0.03480841  0.08888245 -0.08279111 -0.03537029 -0.27922571  0.20285759
 -0.04041531  0.01401394 ...]
std deviations: [3.52161066e-10 3.28248955e-10 3.28931886e-10 3.08632269e-10
 3.08869316e-10 3.55543673e-10 4.0530

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6256.7166613] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6886.18859152] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   67    871 -2.780204815057489e+09 2.1e+00 1.04e+02  9e+01  1e+02 0:00.9
termination on tolfacupx=1000.0 (Mon Apr 13 13:15:17 2020)
final/bestever f-value = -2.639603e+09 -2.780205e+09
incumbent solution: [ 162.45753847 -453.59252412  359.99106326   74.05321665  -25.98986039
 -106.70542094    6.11938797  -14.60812402 ...]
std deviations: [ 97.49593681 104.84158958  96.60735693 103.04911365  94.28971425
  97.10648425  99.41756195  94.60889098 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F3_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F3(X_Values.iloc[i,:25])
    SVM_Fun [i] = F3(X_Values.iloc[i,25:50])
    ELN_Fun [i] = F3(X_Values.iloc[i,50:75])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([1675.22633051]), count=array([1])) 151.03726192440234
SVM
ModeResult(mode=array([1674.7038684]), count=array([1])) 3.955140976365404e-06
ELN
ModeResult(mode=array([3.79850113e+20]), count=array([1])) 8.164452112764524e+40
